In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
!pip install python-dotenv openAI langchain_core langchain_openai langchain langchain-community sentence_transformers rank_bm25

In [ ]:
# <오픈AI 임베딩을 활용한 벡터들 간 코사인 유사도 계산>

# 라이브러리 불러오기
import os
from dotenv import load_dotenv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)

[-0.016639344283517023, -0.021817782168952537, 0.015167301584591451, -0.027232798312025287, -0.036853651802111066, 0.011815774715859829, -0.034540440516195925, -0.006712910854423972, -0.0239601311729469, -0.01679706374282349, -0.008148809859837739, 0.010889175919232344, -0.010613168262429723, -0.022540659764478554, 0.0112440433056882, -0.004941859162137045, 0.012124641291843145, -0.0029457291820003994, 0.008115951871979507, -0.016074184284652546, 0.0013948265205909764, -0.014510139033459433, 0.018229676111261216, -0.012683228948078005, 0.00332852599783412, 0.006433617026306543, 0.00538872910086044, -0.019123417851352933, -0.009193697785283842, -0.0017776234528400052, 0.03451415487096732, -0.016402766025879787, 7.880811401603981e-05, 0.003087018948885903, 0.007366787234241182, -0.005444587959616172, -0.006377758167550811, 0.0033712415683143137, -0.003933115395863982, -0.0022162792438447662, -0.022501231296635633, -0.010396304238713948, 0.011355761023199663, -0.024775012252062918, 0.0132

In [ ]:
# <오픈AI 임베딩을 활용한 문장들 간 코사인 유사도 계산>

data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])
df

,text
0,주식 시장이 급등했어요
1,시장 물가가 올랐어요
2,전통 시장에는 다양한 물품들을 팔아요
3,부동산 시장이 점점 더 복잡해지고 있어요
4,저는 빠른 비트를 좋아해요
5,최근 비트코인 가격이 많이 변동했어요


In [ ]:
# 텍스트를 임베딩 벡터로 변환하는 함수 정의
def get_embedding(text):
  return embeddings.embed_query(text)

# DataFrame의 각 행에 대해 'text' 열의 내용을 임베딩 벡터로 변환
df['embedding'] = df.apply(lambda row: get_embedding(
        row.text,
    ), axis=1)

# 변환된 DataFrame 출력
df

,text,embedding
0,주식 시장이 급등했어요,"[-0.013049525952933459, -0.03346164849092503, ..."
1,시장 물가가 올랐어요,"[-0.00011368328302991353, -0.03533018255334717..."
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009272350082520624, -0.014589923032291684,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.01742896989806365, -0.0031272701334417465,..."
4,저는 빠른 비트를 좋아해요,"[-0.03798272782653637, -0.013369607561132537, ..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.01744061314815701, -0.021532895219966937, ..."


In [ ]:
# 코사인 유사도 계산 함수
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

# 주어진 쿼리와 가장 유사한 상위 3개의 문서를 반환하는 함수
def return_answer_candidate(df, query):
    # 쿼리 텍스트를 임베딩 벡터로 변환
    query_embedding = get_embedding(query)

    # DataFrame의 각 문서 임베딩과 쿼리 임베딩 간의 유사도 계산
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))

    # 유사도가 높은 순으로 정렬하고 상위 3개 문서 선택
    top_three_doc = df.sort_values("similarity", ascending=False).head(3)

    return top_three_doc

# 예시 쿼리로 유사한 문서 검색
sim_result = return_answer_candidate(df, '과일 값이 비싸다.')
sim_result

,text,embedding,similarity
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009272350082520624, -0.014589923032291684,...",0.824431
1,시장 물가가 올랐어요,"[-0.00011368328302991353, -0.03533018255334717...",0.814509
0,주식 시장이 급등했어요,"[-0.013049525952933459, -0.03346164849092503, ...",0.806843


In [ ]:
# <BGE-M3 임베딩을 활용한 문장들 간 코사인 유사도 계산>

# 라이브러리 불러오기
from langchain.embeddings import HuggingFaceBgeEmbeddings

# BGE-M3 모델 초기화
embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

# 데이터셋 생성
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])

# DataFrame의 각 행을 벡터로 변환
df['embedding'] = df['text'].apply(get_embedding)
df

,text,embedding
0,주식 시장이 급등했어요,"[-0.015141432173550129, 0.027917448431253433, ..."
1,시장 물가가 올랐어요,"[0.013636063784360886, 0.05754707381129265, -0..."
2,전통 시장에는 다양한 물품들을 팔아요,"[0.017030559480190277, 0.04437505826354027, -0..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[0.0001659866829868406, 0.06238185241818428, 0..."
4,저는 빠른 비트를 좋아해요,"[0.006794504821300507, 0.04784494638442993, -0..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.0011448436416685581, 0.02486421726644039, ..."


In [ ]:
# 쿼리 예시로 유사 문서 검색
sim_result = return_answer_candidate(df, '과일 값이 비싸다')
sim_result

,text,embedding,similarity
0,주식 시장이 급등했어요,"[-0.015141432173550129, 0.027917448431253433, ...",1.0
5,최근 비트코인 가격이 많이 변동했어요,"[-0.0011448436416685581, 0.02486421726644039, ...",1.0
1,시장 물가가 올랐어요,"[0.013636063784360886, 0.05754707381129265, -0...",-1.0
